# Simualcion Del Colegio manuel cordova galarza
total de docenetes 22
estudiantes masculino 121
estudiantes femeninos 121
total de estudiantes 242
numero de cursos 22
estudiantes por curso 11




In [3]:
import simpy
import random
import matplotlib.pyplot as pp
import math
import pandas as pd

In [4]:
# Num. de Estudiantes en la Intitucion
NUM_ESTUDIANTES = 2000

# Num. de Profesores de la Institucion
NUM_DOCENTES = 200

# Num. de paralelos de la Intitucion
NUM_PARALELOS = 40

# Tiempos

In [5]:
#Tiempo en dias de la simulacion
TMP_DIAS=30

# Tiempo en minutos del mes sin contar los fines de semana
TMP_GENERAL = 28800 

# Tiempo Obligatorio
TMP_RECESO_OBLIGATORIO = 30

# Tiempo dentro de las aulas antes del receso 360/2=180
TMP_CURSO = 180

# Almacenar

In [6]:
#Lista para contagiodos dentro de las pruebas pcr.
list_contagiados_pcr = []

# Lista de estudiantes contagiados
list_contagiados = []

#Lista de estudiantes contagiados por id de curso.
list_contagiados_id = []


# Lista paralelos cerrados por covid
list_paralelos_covid = []

#Dic de estudiantes por curso en la prueba pcr.
dic_estudiantes_pcr = {}

#Aux Estudiantes Sin Contagiar
dic_sin_contagio={}

#Aux game
dic_contagio_dia={}

# Funciones

In [7]:
def foco_contagio(env):
    # Seleccion PCR y Covid
    for cont in range(len(dic_estudiantes_pcr.keys())):
        if(len(list_contagiados_pcr)< int((len(dic_estudiantes_pcr.keys())*2)/100)):
            
            #Definimos el porcentaje de estudiantes
            if((random.randint(1,10))==5):
                
                #Guardamos El Id Del estudiante
                id=dic_estudiantes_pcr[cont+1][1]
                
                #Agregamos al list del contagiados dentro del las pruebas pcr.
                list_contagiados_id.append(id)
                list_contagiados_pcr.append(id)

In [8]:
def foco_contagio_receso(env):

        aux_contagios=0
        if(len(list_contagiados)<=CONTAGIO_RECREO):
            for j in range(len(numEst.keys())):
                if(aux_contagios<2):
                    if(numEst[j+1][0]==0):
                        # Si se cumple la condicion contagiamos a uno aleatoriamente
                        if((random.randint(1,NUM_ESTUDIANTES))==5):
                            # Aumentamos el numero estudiantes contagiados
                            # Asignamos al estudiante el estado: Contagiado
                            # Y guardamos el id del estudiante contagiado
                            id=numEst[j+1][1]
                            list_contagiados.append(id)
                            list_contagiados_id.append(id)
                            aux_contagios=aux_contagios+1
        print('Los Estudiantes ingresan del receso a las: [%s] '%(env.now)) 
        return aux_contagios
    

In [9]:
def pcr(env):
    yield env.timeout(30)
    
    #Guardamos los estudiantes que estan sin contagio.
    aux=0
    for i in range(len(numEst.keys())):
        if(numEst[i+1][0]==0):
            dic_sin_contagio[aux]=numEst[i+1]
            aux=aux+1
    
  
    #Guardamos todos los estudiantes que se realizan al prueba pcr
    aux2=0
    for i in range(int(len(dic_sin_contagio.keys()))):
        if(int(len(dic_estudiantes_pcr.keys()))<=int(len(dic_sin_contagio.keys())*10)/100):
            
            #Definimos el porcentaje de estudiantes
            if((random.randint(1,10))==5):
                dic_estudiantes_pcr[aux2]=dic_sin_contagio[i+1]
                aux2=aux2+1

    foco_contagio(env)
    

In [10]:
def simulacion_colegio(env):
    for i in range(TMP_DIAS):
        print("----------- Ingreso Del Dia [%s] en el Colegio Manuel Cordova ----------"%(i))
        print('Dia [%s] ingresan un total de [%s] '%(i,NUM_EST_VACUNDOS))
        
        yield env.timeout(TMP_CURSO)

        
        print('Los Estudiantes salen al receso a las: [%s] '%(env.now))
        yield env.timeout(TMP_RECESO_OBLIGATORIO)
 
        aux_contagios=foco_contagio_receso(env)
        
    
        yield env.timeout(TMP_CURSO)
        
        
        print("Al terminar el dia existe","\033[1m", aux_contagios ,"\033[0m"," de contagidos")
        dic_contagio_dia[i]=aux_contagios
        print("Cantidad De Contagiados : ","\033[1m", sum(dic_contagio_dia.values()) ,"\033[0m")
        print("----------- Salida Del Colegio Manuel Cordova ----------")
    
    #Obtenemos los resultados del pcr de los estudiantes
    yield env.process(pcr(env))

# Porcentajes

In [11]:
# Sacamos entre 5-10% de estudiantes vacunados
porcentaje=random.randint(90, 95)
NUM_EST_VACUNDOS = int((NUM_ESTUDIANTES*porcentaje)/100)

# Sacamos el 2% de los estudiantes que se  contagian en el recreo 
CONTAGIO_RECREO = int((NUM_EST_VACUNDOS*2)/100)+1  

# Sacamos el 90 % de los profesores
NUM_DOCENTES_VACUNADOS = int((NUM_DOCENTES*90)/100)

# Agrupar estudiantes por Curso

In [12]:
#Diccionario de los [estudiante]=curso
est_cursos = {}
for i in range(1,NUM_EST_VACUNDOS):
    est_cursos[i] = i,random.randint(1,40)

#Diccionario [estudiantes]=contagiado/no contagiado
numEst = {}
for i in range(1,NUM_EST_VACUNDOS):
    numEst[i]= 0,i


# Ejecutar

In [13]:
env=simpy.Environment()
env.process(simulacion_colegio(env))

# Ejecutanmos la simulacion en Funcion Del Tiempo General
env.run(until = TMP_GENERAL)


----------- Ingreso Del Dia [0] en el Colegio Manuel Cordova ----------
Dia [0] ingresan un total de [1900] 
Los Estudiantes salen al receso a las: [180] 
Los Estudiantes ingresan del receso a las: [210] 
Al terminar el dia existe  1   de contagidos
Cantidad De Contagiados :   1 
----------- Salida Del Colegio Manuel Cordova ----------
----------- Ingreso Del Dia [1] en el Colegio Manuel Cordova ----------
Dia [1] ingresan un total de [1900] 
Los Estudiantes salen al receso a las: [570] 
Los Estudiantes ingresan del receso a las: [600] 
Al terminar el dia existe  0   de contagidos
Cantidad De Contagiados :   1 
----------- Salida Del Colegio Manuel Cordova ----------
----------- Ingreso Del Dia [2] en el Colegio Manuel Cordova ----------
Dia [2] ingresan un total de [1900] 
Los Estudiantes salen al receso a las: [960] 
Los Estudiantes ingresan del receso a las: [990] 
Al terminar el dia existe  1   de contagidos
Cantidad De Contagiados :   2 
----------- Salida Del Colegio Manuel Cordo

# Simulacion Pygame

In [12]:
import random
import pygame



#Agrupamos los Cursos que se deben cerrar por covid

for i in range(len(list_contagiados_id)):
    for j in range(int(len(est_cursos.keys()))):
        if(est_cursos[j+1][0]==list_contagiados_id[i]):
            list_paralelos_covid.append(est_cursos[j+1][1])

list_paralelos_covid = list(dict.fromkeys(list_paralelos_covid))


pygame.init()
screen = pygame.display.set_mode((700, 1000))
clock = pygame.time.Clock()





counter, text = 0, '0'.rjust(3)
pygame.time.set_timer(pygame.USEREVENT, 1000)
font = pygame.font.SysFont('Consolas', 15)
fontitulo=pygame.font.SysFont('Consolas', 30)
font2 = pygame.font.SysFont('Consolas', 10)

verde=(0,255,0)
rojo=(255,0,0)
azul=(0,0,255)

for i in range(1000):
    X = random.randint(0, 700)
    Y = random.randint(0, 700)
    #DIBUJAMOS CIRCULO.
    pygame.draw.circle(screen, verde, (X,Y),5,0)

    
    
#informacion para game
dic_contagiados=dic_contagio_dia
 
cont_contagiado=0    
while True:
    for e in pygame.event.get():     
        if e.type == pygame.USEREVENT: 
            counter += 1
            text = str(counter).rjust(3) if counter < 30 else 'Fin De Simulacion'
        #if e.type == pygame.QUIT: break
        
        if e.type == pygame.KEYDOWN:
            if e.key == pygame.K_LEFT:
                print("Detener")
                pygame.quit()
                quit()
            if e.key == pygame.K_SPACE:
                print("Se presiono")
                pygame.time.wait(1000)

                
    else:
        screen.fill((255, 255, 255))
        
        screen.blit(fontitulo.render("------- Colegio Manuel Cordova Galarza ----------", True,azul), (32, 725))
        
        screen.blit(font.render('DIA:'+text, True, (0, 0, 0)), (32, 760))
        
        screen.blit(font.render('Estudiantes matriculados '+str(NUM_ESTUDIANTES)+' de los cuales ingresan solo :'+str(NUM_EST_VACUNDOS)+'de estudiantes.', True, (0, 0, 0)), (32, 790))
        
        screen.blit(font.render('Docentes de la intitucion '+str(NUM_DOCENTES)+' de los cuales ingresan solo :'+str(NUM_DOCENTES_VACUNADOS) +'de docentes.', True, (0, 0, 0)), (32, 820))
        
        screen.blit(font.render('Cantidad De Paralelos Por Cerrar:'+str(len(list_paralelos_covid)), True, (0, 0, 0)), (32, 850))
        
        
        screen.blit(font2.render('Curso que se deben cerrar:', True, (0, 0, 0)), (32, 880))
        screen.blit(font2.render(str(list_paralelos_covid), True, (0, 0, 0)), (32, 900))
        
        for i in range(1000):
            X = random.randint(0, 700)
            Y = random.randint(0, 700)
            #DIBUJAMOS CIRCULO.
            pygame.draw.circle(screen, verde, (X,Y),5,0)
        
        #print(text)
        #if int(text)==dic_contagiados[int(text)]:
            #print("ingreso")
        #print("dia",int(text),"graficar puntos:",dic_contagiados.get(int(text)))
        try:
            for i in range(dic_contagiados.get(int(text))):
                X = random.randint(0, 700)
                Y = random.randint(0, 700)
                pygame.draw.circle(screen, rojo, (X,Y),5,0)
                cont_contagiado=cont_contagiado+1

            pygame.display.flip()

            clock.tick(60)
            continue
        except:
            from tkinter import messagebox
            messagebox.showinfo(message="Fin De La simulacion con un total de contagiados de: "+str(sum(dic_contagiados.values())), title="Simulacion")
    break

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Detener


error: display Surface quit